Definición de librerias que se utilizaran

In [13]:
import sys
import warnings
import os
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
#from __future__ import unicode_literals
warnings.filterwarnings("ignore")
#Aquí poner la dirección donde tienen la carpeta
sys.path.append('/content/drive/MyDrive/mce/Cursos/dl/Modelo/Entrenamiento')
path = '/content/drive/MyDrive/mce/Cursos/dl/Modelo/Entrenamiento'

#sys.path.append('/content/drive/MyDrive/Modelo/Entrenamiento')
#path = '/content/drive/MyDrive/Modelo/Entrenamiento'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/Modelo/Entrenamiento')
!pwd

In [ ]:
os.chdir(path)

In [15]:
import argparse
import json
import tensorflow as tf
from transformer import Transformer, default_hparams
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
import numpy as np

In [7]:
print(json.__version__)
print(tf.__version__)
print(pd.__version__)
print()

2.0.9
2.14.0


NameError: ignored

In [ ]:
#Permite crecimiento dinámico de memoria en GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false'
#Orden de dispositivos CUDA
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'  # see issue #152
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda'

Obtención del dataset

A continuación se tiene que definir los directorios

In [ ]:
def parse_function(filename, texts_inputs, texts_labels):
    # Read entire contents of image
    image_string = tf.io.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.io.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Resize image with padding to 244x244
    image = tf.image.resize_with_pad(image, 224, 224, method=tf.image.ResizeMethod.BILINEAR)

    return (image, texts_inputs), texts_labels


def augmentation_fn(inputs, texts_labels):
    # Random left-right flip the image
    image, texts_inputs = inputs
    image = tf.image.random_flip_left_right(image)

    # Random brightness, saturation and contrast shifting
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return (image, texts_inputs), texts_labels


def make_grayscale_fn(inputs, texts_labels):
    # Convert image to grayscale
    image, texts_inputs = inputs
    image = tf.image.rgb_to_grayscale(image)

    return (image, texts_inputs), texts_labels

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            'd_model': str(int(self.d_model)),
            'warmup_steps': str(int(self.warmup_steps)),
        }


def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_object(label, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match) / tf.reduce_sum(mask)

In [ ]:
def get_mimic_dataset(csv_root,vocab_root,mimic_root,
                      max_length=128,batch_size=16,n_threads=16,
                      buffer_size=10000,mode='train',unsure=1):

    assert mode in ['train', 'validate', 'test']
    assert unsure in [0, 1]
    # Load Byte-Level BPE Tokenizer with mimic vocabulary
    tokenizer = ByteLevelBPETokenizer(
        os.path.join(vocab_root, 'mimic-vocab.json'),
        os.path.join(vocab_root, 'mimic-merges.txt'),
    )
    # Read MIMIC_AP_PA_{mode}.csv file and set unsure values (-1) to 0 or 1
    csv_file        = os.path.join(csv_root, f'MIMIC_AP_PA_{mode}.csv')
    replacements    = {float('nan'): 0, -1.0: unsure}
    reports         = pd.read_csv(csv_file).replace(replacements).values

    image_paths     = [os.path.join(mimic_root, path) for path in reports[:, 0]]
    texts           = reports[:, 17]
    labels          = np.uint8(reports[:, 3:16])

    # Tokenize reports
    texts_tokenized = tokenizer.encode_batch(list(texts))
    texts_tokenized = [[tokenizer.token_to_id('<s>')] +
                             seq.ids +
                             [tokenizer.token_to_id('</s>')]
                             for seq in texts_tokenized]
    texts_tokenized = tf.keras.preprocessing.sequence.pad_sequences(
        texts_tokenized, maxlen=max_length, dtype='int32', padding='post', truncating='post')

    texts_tokenized = texts_tokenized[:, :(max_length + 1)]
    texts_inputs = texts_tokenized[:, :-1]  # Drop the [END] tokens
    texts_labels = texts_tokenized[:, 1:]  # Drop the [START] tokens

    # Create Tensorflow dataset (image, text) pair
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, texts_inputs, texts_labels))
    if mode == 'train':
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.map(parse_function, num_parallel_calls=n_threads)
    if mode == 'train':
        dataset = dataset.map(augmentation_fn, num_parallel_calls=n_threads)
    dataset = dataset.map(make_grayscale_fn, num_parallel_calls=n_threads)
    dataset = dataset.batch(batch_size)
    # dataset = dataset.prefetch(buffer_size)
    return dataset, tokenizer

In [ ]:
args = {'csv_root':path+'/mimic',
        'vocab_root':path+'/mimic',
        'mimic_root':path+'/',
        'model_name':'RATCHETESP',
        'model_params':path+'/hparams.json',
        'classifier_weights':path+'/checkpoints/RATCHETESP.tf',
        'n_epochs':20,
        'init_lr':None,
        'batch_size':16,
        'seed':42}
hparams = default_hparams()

In [ ]:
'''if args['model_params']:
        with open(args['model_params']) as json_file:
          print(json_file)
          hparams_from_file = json.load(json_file)
          hparams.update((k, hparams_from_file[k])
              for k in set(hparams_from_file).intersection(hparams))'''

    # Set tensorflow random seed
tf.random.set_seed(args['seed'])

In [ ]:
train_batches, tokenizer = get_mimic_dataset(args['csv_root'], args['vocab_root'], args['mimic_root'])

In [ ]:
val_batches, _ = get_mimic_dataset(args['csv_root'], args['vocab_root'], args['mimic_root'],mode='validate',batch_size=args['batch_size'])

No quieres verla mejor mañana para que no te este tirando?
pq va aempezar a decir que me interrumpiste

In [ ]:
csv_logger_callback = tf.keras.callbacks.CSVLogger('checkpoints/training.log')
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/'+args['model_name']+'.tf',
    save_weights_only=True,
    monitor='val_masked_accuracy',
    mode='max',
    save_best_only=True)

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    learning_rate = args['init_lr'] if args['init_lr'] is not None else \
          CustomSchedule(hparams['d_model'])

    optimizer = tf.keras.optimizers.Adam(
        learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    transformer = Transformer(
        num_layers=hparams['num_layers'],
        d_model=hparams['d_model'],
        num_heads=hparams['num_heads'],
        dff=hparams['dff'],
        target_vocab_size=tokenizer.get_vocab_size(),
        dropout_rate=hparams['dropout_rate'],
        input_shape=(224, 224, 1))
        #classifier_weights=args['classifier_weights'])
    transformer.load_weights(args['classifier_weights'])
    transformer.compile(
        loss=masked_loss,
        optimizer=optimizer,
        metrics=[masked_accuracy],
    )

Number of devices: 1
2023-12-09 00:03:29.231165: I No Pretrained DenseNet-121 weights specified


In [ ]:
transformer.fit(
    train_batches,
    epochs=args['n_epochs'],
    validation_data=val_batches,
    callbacks=[model_checkpoint_callback, csv_logger_callback]
)

Epoch 1/20
1451/1451 [==============================] - 3259s 2s/step - loss: 1.5333 - masked_accuracy: 0.6136 - val_loss: 2.2792 - val_masked_accuracy: 0.5274
Epoch 2/20
1451/1451 [==============================] - 1313s 904ms/step - loss: 1.6019 - masked_accuracy: 0.5981 - val_loss: 2.2560 - val_masked_accuracy: 0.5291
Epoch 3/20
1451/1451 [==============================] - 1245s 858ms/step - loss: 1.5604 - masked_accuracy: 0.6067 - val_loss: 2.2826 - val_masked_accuracy: 0.5260
Epoch 4/20
1451/1451 [==============================] - 1174s 808ms/step - loss: 1.5229 - masked_accuracy: 0.6137 - val_loss: 2.2898 - val_masked_accuracy: 0.5321
Epoch 5/20
1451/1451 [==============================] - 1207s 831ms/step - loss: 1.4895 - masked_accuracy: 0.6205 - val_loss: 2.3151 - val_masked_accuracy: 0.5253
Epoch 6/20
1451/1451 [==============================] - 1169s 805ms/step - loss: 1.4650 - masked_accuracy: 0.6258 - val_loss: 2.3190 - val_masked_accuracy: 0.5314
Epoch 7/20
1451/1451 [===

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-3df87481b532>", line 1, in <cell line: 1>
    transformer.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py", line 60, in quick_execute
    tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
tensorflow.python.framework.errors_impl.FailedPreconditionError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) FAILED_PRECONDITION:  /content/drive/MyDrive/Modelo/Entrenamiento/files/p